In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from utils import run_ML

In [3]:
# !pip install tldextract

In [4]:
data_dir = "data/URLdatasetX2_1.csv"
df = pd.read_csv(data_dir,index_col=0)

In [5]:
df.shape, df.head(2)

((2802, 2),
                                           url        type
 0       http://www.crestonwood.com/router.php  legitimate
 1  http://vamoaestudiarmedicina.blogspot.com/  legitimate)

In [6]:
# smalldata = df.sample(n = 20000, random_state=1)
smalldata = df

In [7]:
if 'PhiUSIIL' in data_dir:
    urldata = np.array(smalldata[['URLLength', 'DomainLength', 'IsDomainIP', \
           'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb',\
           'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation',\
           'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL',\
           'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',\
           'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',\
           'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS']])
    labels = smalldata.iloc[:,-1].values
    run_ML(urldata, labels, "PhiUSIILmock", "standard")
if 'URLdatasetX2' in data_dir:
    labels = smalldata.iloc[:,-1].values
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

In [8]:
# labels

### Feature extraction

In [9]:
from utils import extract_features

In [10]:
# Example usage:
url = "http://www.example.com/path/to/==file.html"
url_features = extract_features(url)
print(url_features)

{'domain': 'www.example.com', 'num_subdomains': 2, 'contains_ip': 0, 'path_length': 20, 'num_path_segments': 3, 'uses_https': 0, 'file_extension': 'html', 'count_special_characters': 11, 'count_non_alphanumeric_characters': 11, 'TLD': 'com', 'count_obfuscated_characters': 0, 'letter_ratio_in_url': 0.7380952380952381, 'digit_ratio_in_url': 0.0, 'count_equals_in_url': 2, 'NoOfAmpersandInURL': 0, 'CharContinuationRate': 0.11904761904761904, 'ratio_obfuscated_characters': 0.0, 'NoOfQMarkInURL': 0}


In [11]:
# print(url_features.keys())

In [12]:
phish_url = []
for link in list(smalldata.iloc[:,0]):
    url_features = extract_features(link)
    phish_url.append(list(url_features.values())[1:])

In [13]:
phish_url_df = pd.DataFrame(phish_url, columns = list(url_features.keys())[1:])

In [14]:
# phish_url_df.head(2)

In [15]:
phish_url_df.iloc[:,5] = pd.Categorical(phish_url_df.iloc[:,5]).codes
phish_url_df.iloc[:,8] = pd.Categorical(phish_url_df.iloc[:,8]).codes

In [16]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,2,0,11,1,0,47,7,7,38,0,0.810811,0.0,0,0,0.135135,0.0,0
1,2,0,1,1,0,0,6,6,38,0,0.857143,0.0,0,0,0.047619,0.0,0


In [17]:
run_ML(phish_url_df, labels, "URLdatasetX2", "manual")

Run:  0 , fold:  0
Train freq:  [1630, 611]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [1617, 624]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [1602, 640]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [1645, 597]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [1622, 620]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.73 0.86]


In [18]:
## Numerical features
from utils import extract_numerical_features
phish_url = []
for link in list(smalldata.iloc[:,0]):
    url_features = extract_numerical_features(link)
    phish_url.append(list(url_features.values()))
run_ML(np.array(phish_url), labels, "URLdatasetX2", "manual_numerical")

Run:  0 , fold:  0
Train freq:  [1630, 611]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [1617, 624]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [1602, 640]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [1645, 597]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [1622, 620]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.73 0.83]


In [21]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

In [ ]:
hyperlink_features = np.zeros((smalldata.shape[0], 48))
enum = 0;
for url in list(smalldata.iloc[396:,0]):
    # url = 'https://polarklimatsgserver.blogspot.com/'
    try:    
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        urls = []
        count = 0;
        for link in soup.find_all('a'):
            # print(link.get('href'))
            weblink = link.get('href')
            if (weblink is not None) and ('http' in weblink):
                urls.append(weblink)
            count += 1
            if count > 50:
                break
        # extract numerical features in link
        if len(urls) > 0:
            hyperlink_data = []
            for link in urls:
                try:
                    url_features = extract_numerical_features(link)
                    datalinkssss = list(url_features.values())
                except ValueError as ve:
                    datalinkssss = list(np.zeros(15))
                hyperlink_data.append(datalinkssss)
            hyperlink_data = np.array(hyperlink_data)
            hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),
                                  hyperlink_data.min(axis=0),hyperlink_data.max(axis=0), hyperlink_data.mean(axis=0)))
        else:
            hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),np.zeros(45)))
    
    except ConnectionError as e:
        print("No rep", end = ',')
        hyper_np = np.zeros(48)
    hyperlink_features[enum, :] = hyper_np
    print(enum, end =',')
    enum += 1

0,1,2,3,4,5,6,7,8,9,10,11,No rep,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275

In [ ]:
np.save('data/hyperlink_features.npy', hyperlink_features) # save
np.save('data/phish_url_df.npy', phish_url_df.to_numpy()) # save

In [ ]:
run_ML(np.concatenate((phish_url_df.to_numpy(), hyperlink_features),axis=1), labels, "URLdatasetX2", "hyperlinks_features")